In [1]:
import json
import os
import time
import psycopg2
from psycopg2 import sql
import math
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime, timedelta
import numpy as np

load_dotenv("../../.buenoshops")
BUENO_ACCESS = os.getenv("BUENO_ACCESS")
BUENO_SELLER_ID = os.getenv("BUENO_SELLER_ID")

load_dotenv("../../.musicalcris")
CRIS_ACCESS = os.getenv("CRIS_ACCESS")
CRIS_SELLER_ID = os.getenv("CRIS_SELLER_ID")

load_dotenv("../../.mcenter")
MCENTER_ACCESS = os.getenv("MCENTER_ACCESS")
MCENTER_SELLER_ID = os.getenv("MCENTER_SELLER_ID")

print(f"MCENTER_ACCESS {MCENTER_ACCESS}")
print(f"MCENTER_SELLER_ID {MCENTER_SELLER_ID}\n")

print(f"CRIS_ACCESS {CRIS_ACCESS}")
print(f"CRIS_SELLER_ID {CRIS_SELLER_ID}\n")

print(f"BUENO_ACCESS {BUENO_ACCESS}")
print(f"BUENO_SELLER_ID {BUENO_SELLER_ID}\n")

HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": HOST,
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

MCENTER_ACCESS APP_USR-6628000663056293-121116-25a2cf9e1d6aeb2395e35a8d00e0afd3-233632476
MCENTER_SELLER_ID 233632476

CRIS_ACCESS APP_USR-2611836769858126-121116-368411ccf4ab9092e12b0c10b5fc652f-681023789
CRIS_SELLER_ID 681023789

BUENO_ACCESS APP_USR-1402424125091767-121116-6b777263d63f35d7763f7d3b29b5d9dc-648304687
BUENO_SELLER_ID 648304687



In [22]:
from dotenv import find_dotenv, load_dotenv

env_path

'c:\\Users\\couti\\projetos\\mcenter_streamlit\\Notebooks\\populando_tabela\\.env'

In [34]:
def update_config_file(file_path, new_values):
    with open(file_path, "r") as config_file:
        lines = config_file.readlines()

    with open(file_path, "w") as config_file:
        for line in lines:
            for key, value in new_values.items():
                if line.startswith(f"{key}="):
                    print(f"Updating {key} to {value}")
                    line = f"{key}='{value}'\n"
                    break
            config_file.write(line)


# Example usage
new_values = {
    "ACCESS_TOKEN_MCENTER": "xxxxx",
    "REFRESH_TOKEN_MCENTER": "xxxx",
}

env_path = find_dotenv()
print(env_path)
update_config_file(env_path, new_values)

c:\Users\couti\projetos\mcenter_streamlit\.env
Updating ACCESS_TOKEN_MCENTER to xxxxx
Updating REFRESH_TOKEN_MCENTER to xxxx


In [2]:
# Funções para consultar DFs
def condf(df, coluna, valor):
    """
    Consulta um DataFrame com base em uma coluna e valor específicos.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna: Nome da coluna para a condição de consulta.
    - valor: Valor desejado na coluna.

    Retorna:
    Um DataFrame contendo apenas as linhas que atendem à condição.
    """
    resultado = df[df[coluna] == valor]
    return resultado


def condf_date(df, coluna_data, data_pesquisada):
    """
    Consulta um DataFrame com base em uma coluna de datas.

    Parâmetros:
    - df: DataFrame a ser consultado.
    - coluna_data: Nome da coluna de datas.
    - data_pesquisada: Data desejada para a consulta.

    Retorna:
    Um DataFrame contendo apenas as linhas que correspondem à data pesquisada.
    """
    resultado = df[pd.to_datetime(df[coluna_data]).dt.date == data_pesquisada]
    return resultado

### Consultando códigos de anúncio(ml_code) onde 'logistic_type=fulfillment'

In [9]:
base_url_BUENO = f"https://api.mercadolibre.com/users/{BUENO_SELLER_ID}/items/search?logistic_type=fulfillment"
base_url_CRIS = f"https://api.mercadolibre.com/users/{CRIS_SELLER_ID}/items/search?logistic_type=fulfillment"
base_url_MCENTER = f"https://api.mercadolibre.com/users/{MCENTER_SELLER_ID}/items/search?logistic_type=fulfillment"

params = {
    "limit": 100,
    "offset": 0,
}

# headers = {"Authorization": f"Bearer {BUENO_ACCESS}"}
headers = {"Authorization": f"Bearer {CRIS_ACCESS}"}
# headers = {"Authorization": f"Bearer {MCENTER_ACCESS}"}

json_list = []

try:
    while True:
        # response = requests.get(base_url_BUENO, headers=headers, params=params)
        response = requests.get(base_url_CRIS, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        print(data)
        if "results" in data:
            json_list.extend(data["results"])
            print(data["results"])
        else:
            break

        # Verifique se há mais páginas
        if "paging" in data:
            total_data = data["paging"].get("total")

            total_pages = math.ceil(total_data / params["limit"])
            print(f"Total de páginas a serem processadas: {total_pages}")
            print(f'Offset atual: {params["offset"]}')

            if params["offset"] >= total_pages * params["limit"]:
                break

            params["offset"] += params["limit"]
        else:
            break

except requests.exceptions.RequestException as req_err:
    # print(f"Erro ao fazer a requisição para {base_url_BUENO}: {req_err}")
    print(f"Erro ao fazer a requisição para {base_url_CRIS}: {req_err}")
except Exception as e:
    print(f"Erro não esperado: {e}")

print(f"Total esperado de dados: {total_data}")
print(f"Total de dados coletados: {len(json_list)}")

{'seller_id': '681023789', 'results': ['MLB2077624899', 'MLB2077566813', 'MLB3642965726', 'MLB3321195945', 'MLB3321186153', 'MLB3321181307', 'MLB3321175509', 'MLB3321160629', 'MLB3321122477', 'MLB3661055492', 'MLB3661047876', 'MLB4097062232', 'MLB3467798335', 'MLB4099722814', 'MLB4099639536', 'MLB4099590706', 'MLB4099564966', 'MLB4099421026', 'MLB4099400438', 'MLB4099382194', 'MLB4099379150', 'MLB4099376828', 'MLB4099364154', 'MLB3468588713', 'MLB3468588049', 'MLB3468571827', 'MLB3468508903', 'MLB3468507631', 'MLB3468483295', 'MLB3468445013', 'MLB3468355335', 'MLB3468342661', 'MLB3468337879', 'MLB3468334819', 'MLB3468308899', 'MLB3468277673', 'MLB3468270317', 'MLB3468268905', 'MLB3468264183', 'MLB3468232887', 'MLB4102601126', 'MLB4102433058', 'MLB4102407438', 'MLB4102391716', 'MLB4102387026', 'MLB4102304362', 'MLB4102267624', 'MLB3469928691', 'MLB3469801625', 'MLB3469766071', 'MLB3469737689', 'MLB3469722247', 'MLB3469668577', 'MLB3469635843', 'MLB3469580967', 'MLB4107101184', 'MLB41070

### Pesquisando itens onde 'logistic_type=fulfillment', usando o ml_code

In [10]:
json_list_item = []

c = 1
for item in json_list:
    base_url = f"https://api.mercadolibre.com/items/{item}"

    # headers = {"Authorization": f"Bearer {BUENO_ACCESS}"}
    headers = {"Authorization": f"Bearer {CRIS_ACCESS}"}
    # headers = {"Authorization": f"Bearer {MCENTER_ACCESS}"}

    t = len(json_list)
    print(item)
    print(f"{c}/{t}")
    c += 1

    try:
        response = requests.get(base_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        json_list_item.append(data)
        print(f"Tamanho da nova lista: {len(json_list_item)}/{t}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao obter dados para o item {item}: {e}")

    # Se c for um múltiplo de 50, aguarde 1 minuto
    if c % 50 == 0:
        print("Esperando 1 minuto...")
        time.sleep(60)

print(f"Tamanho da lista de itens: {len(json_list_item)}")

MLB2077624899
1/107
Tamanho da nova lista: 1/107
MLB2077566813
2/107
Tamanho da nova lista: 2/107
MLB3642965726
3/107
Tamanho da nova lista: 3/107
MLB3321195945
4/107
Tamanho da nova lista: 4/107
MLB3321186153
5/107
Tamanho da nova lista: 5/107
MLB3321181307
6/107
Tamanho da nova lista: 6/107
MLB3321175509
7/107
Tamanho da nova lista: 7/107
MLB3321160629
8/107
Tamanho da nova lista: 8/107
MLB3321122477
9/107
Tamanho da nova lista: 9/107
MLB3661055492
10/107
Tamanho da nova lista: 10/107
MLB3661047876
11/107
Tamanho da nova lista: 11/107
MLB4097062232
12/107
Tamanho da nova lista: 12/107
MLB3467798335
13/107
Tamanho da nova lista: 13/107
MLB4099722814
14/107
Tamanho da nova lista: 14/107
MLB4099639536
15/107
Tamanho da nova lista: 15/107
MLB4099590706
16/107
Tamanho da nova lista: 16/107
MLB4099564966
17/107
Tamanho da nova lista: 17/107
MLB4099421026
18/107
Tamanho da nova lista: 18/107
MLB4099400438
19/107
Tamanho da nova lista: 19/107
MLB4099382194
20/107
Tamanho da nova lista: 20/10

In [12]:
# Salvando em um arquivo
file_path_bueno = "../../Data/Output/lista_itens_bueno.json"
file_path_cris = "../../Data/Output/lista_itens_cris.json"
file_path_mcenter = "../../Data/Output/lista_itens_mcenter.json"

In [13]:
# Escrever a lista em um arquivo JSON
# with open(file_path_bueno, "w") as file:
with open(file_path_cris, "w") as file:
    # with open(file_path_mcenter, "w") as file:
    json.dump(json_list_item, file)

In [14]:
file_path_bueno = "../../Data/Output/lista_itens_bueno.json"
file_path_cris = "../../Data/Output/lista_itens_cris.json"
file_path_mcenter = "../../Data/Output/lista_itens_mcenter.json"

# with open(file_path_bueno, "r") as file:
with open(file_path_cris, "r") as file:
    # with open(file_path_mcenter, "r") as file:
    json_list_item = json.load(file)

print(f"Tamanho da lista de itens: {len(json_list_item)}")
print(json_list_item)

Tamanho da lista de itens: 107
[{'id': 'MLB2077624899', 'site_id': 'MLB', 'title': 'Mini Acordeon Instrumento Musical Crianças Verde/vermelho', 'seller_id': 681023789, 'category_id': 'MLB72749', 'user_product_id': None, 'official_store_id': None, 'price': 324.73, 'base_price': 324.73, 'original_price': None, 'inventory_id': None, 'currency_id': 'BRL', 'initial_quantity': 3, 'available_quantity': 3, 'sold_quantity': 0, 'sale_terms': [{'id': 'WARRANTY_TYPE', 'name': 'Tipo de garantia', 'value_id': '2230280', 'value_name': 'Garantia do vendedor', 'value_struct': None, 'values': [{'id': '2230280', 'name': 'Garantia do vendedor', 'struct': None}], 'value_type': 'list'}, {'id': 'WARRANTY_TIME', 'name': 'Tempo de garantia', 'value_id': None, 'value_name': '30 dias', 'value_struct': {'number': 30, 'unit': 'dias'}, 'values': [{'id': None, 'name': '30 dias', 'struct': {'number': 30, 'unit': 'dias'}}], 'value_type': 'number_unit'}], 'buying_mode': 'buy_it_now', 'listing_type_id': 'gold_special', 

### Construindo tabela

In [15]:
df = pd.DataFrame(json_list_item)

print(f"Tamanho do dataframe de itens: {df.shape}")
df.sample()

Tamanho do dataframe de itens: (107, 59)


,id,site_id,title,seller_id,category_id,user_product_id,official_store_id,price,base_price,original_price,...,parent_item_id,differential_pricing,deal_ids,automatic_relist,date_created,last_updated,health,catalog_listing,item_relations,channels
74,MLB4109620878,MLB,Violão Eletroacústica Tagima Memphis Md 18 Par...,681023789,MLB3016,None,None,605.0,605.0,None,...,None,None,[],False,2023-10-10T14:52:16.000Z,2023-12-08T10:48:54.465Z,NaN,True,[],[marketplace]


#### attributes: SELLER_SKU

In [16]:
resultados_attributes = []

for item in json_list_item:
    # Extrair os valores desejados
    first_id = item["id"]
    inventory_id = item["inventory_id"]
    variations = item["variations"]
    status = item["status"]
    catalog_product_id = item["catalog_product_id"]
    seller_custom_field = item["seller_custom_field"]
    catalog_listing = item["catalog_listing"]
    logistic_type = item["shipping"]["logistic_type"]
    item_relations = item["item_relations"]

    # Procurar em "attributes" onde "id" é "SELLER_SKU"
    seller_sku_entry = next(
        (attr for attr in item["attributes"] if attr["id"] == "SELLER_SKU"), None
    )

    # Obter os valores de "value_name" e "value_id" se a entrada existir, caso contrário, definir como None
    attribute_value_name = seller_sku_entry["value_name"] if seller_sku_entry else None
    attribute_value_id = seller_sku_entry["value_id"] if seller_sku_entry else None

    # attribute_value_name = item["attributes"][0]["value_name"]
    # attribute_value_id = item["attributes"][0]["value_id"]

    # Adicionar os resultados_attributes à lista
    resultados_attributes.append(
        {
            "ml_code": first_id,
            "inventory_id": inventory_id,
            # "logistic_type": logistic_type,
            # "sku": attribute_value_name,
            "status": status,
            "variations": variations,
            # "attribute_value_id": attribute_value_id,
            # "catalog_product_id": catalog_product_id,
            # "seller_custom_field": seller_custom_field,
            "catalog_listing": catalog_listing,
            # "item_relations": item_relations
        }
    )

# Exibir os resultados
print(resultados_attributes)
df_sku = pd.DataFrame(resultados_attributes)
print(df_sku.shape)
df_sku.sample()

[{'ml_code': 'MLB2077624899', 'inventory_id': None, 'status': 'active', 'variations': [{'id': 173883881289, 'price': 324.73, 'attribute_combinations': [{'id': 'COLOR', 'name': 'Cor', 'value_id': None, 'value_name': 'Vermelho-Verde', 'values': [{'id': None, 'name': 'Vermelho-Verde', 'struct': None}], 'value_type': 'string'}], 'available_quantity': 3, 'sold_quantity': 0, 'sale_terms': [], 'picture_ids': ['660379-MLB48136983769_112021', '899057-MLB48136997637_112021', '697431-MLB48167981882_112021', '838690-MLB48136350502_112021', '975996-MLB48137025511_112021', '924352-MLB48136387284_112021'], 'seller_custom_field': None, 'catalog_product_id': None, 'inventory_id': 'KGZZ89270', 'item_relations': [], 'user_product_id': None}], 'catalog_listing': False}, {'ml_code': 'MLB2077566813', 'inventory_id': 'GOYP17092', 'status': 'active', 'variations': [], 'catalog_listing': False}, {'ml_code': 'MLB3642965726', 'inventory_id': 'GXOM30321', 'status': 'active', 'variations': [], 'catalog_listing': F

,ml_code,inventory_id,status,variations,catalog_listing
65,MLB3472524897,None,active,"[{'id': 180196117171, 'price': 209, 'attribute...",False


#### variations: variation_id,  attribute_combination: value_id, value_name, seller_sku ,inventory_id 

In [17]:
resultados_variations = []

for item in json_list_item:
    # Extrair os valores comuns para cada item
    first_id = item.get("id")
    inventory_id = item.get("inventory_id")
    logistic_type = item.get("shipping", {}).get("logistic_type")

    # Iterar sobre cada variação no item
    for variacao in item.get("variations", []):
        # Extrair os valores específicos para cada variação
        variation_id = variacao.get("id")
        variation_seller_sku = variacao.get("seller_custom_field")
        variation_inventory_id = variacao.get("inventory_id")
        attribute_combination = variacao.get("attribute_combinations", [{}])[0]
        value_id = attribute_combination.get("value_id")
        value_name = attribute_combination.get("value_name")
        item_relations = attribute_combination.get("item_relations", [{}])[0]

        # Adicionar os resultados_variations à lista
        resultados_variations.append(
            {
                "ml_code": first_id,
                "inventory_id": inventory_id,
                # "logistic_type": logistic_type,
                "variation_id": variation_id,
                # "value_id": value_id,
                "value_name": value_name,
                # "var_seller_sku": variation_seller_sku,
                "variation_inventory_id": variation_inventory_id,
                # "item_relations":item_relations,
            }
        )

# Exibir os resultados_variations
print(resultados_variations)
df_variations = pd.DataFrame(resultados_variations)
print(df_variations.shape)
df_variations.sample()

[{'ml_code': 'MLB2077624899', 'inventory_id': None, 'variation_id': 173883881289, 'value_name': 'Vermelho-Verde', 'variation_inventory_id': 'KGZZ89270'}, {'ml_code': 'MLB3321195945', 'inventory_id': None, 'variation_id': 177738888404, 'value_name': 'Preto', 'variation_inventory_id': 'RWRR30846'}, {'ml_code': 'MLB3321181307', 'inventory_id': None, 'variation_id': 178271790957, 'value_name': 'Mahogany oil', 'variation_inventory_id': 'MMFT30494'}, {'ml_code': 'MLB3321175509', 'inventory_id': None, 'variation_id': 178271891283, 'value_name': 'Natural', 'variation_inventory_id': 'DGMW97350'}, {'ml_code': 'MLB3467798335', 'inventory_id': None, 'variation_id': 178909450832, 'value_name': 'Walnut flat', 'variation_inventory_id': 'LLDE88401'}, {'ml_code': 'MLB4099639536', 'inventory_id': None, 'variation_id': 180159251515, 'value_name': 'Preta', 'variation_inventory_id': 'VHRN90158'}, {'ml_code': 'MLB4099382194', 'inventory_id': None, 'variation_id': 180158472193, 'value_name': 'Black night', '

,ml_code,inventory_id,variation_id,value_name,variation_inventory_id
17,MLB4102433058,None,178919001716,110V/220V,TZFM91331


In [18]:
df_variations["variation_inventory_id"].value_counts()

variation_inventory_id
KGZZ89270    1
CPRQ90324    1
NWDU90934    1
LROT89023    1
QHYZ00893    1
KHFD90751    1
UORD89925    1
XDIY00082    1
QNYE96569    1
SAGY91768    1
NYZA98888    1
TGWN96650    1
GGHX00903    1
MDDX95988    1
VRME00762    1
LDAC97015    1
IIKC90307    1
TZFM91331    1
RWRR30846    1
OQSJ90492    1
MMFT30494    1
DGMW97350    1
LLDE88401    1
VHRN90158    1
FMUV90638    1
NJXA90250    1
ACBX96049    1
AGYY89844    1
GNAY89939    1
WGDP88625    1
SSIH91035    1
WCTG88354    1
KKGW88137    1
VLWK91251    1
WYGR96303    1
Name: count, dtype: int64

In [19]:
df_variations["inventory_id"].value_counts()

Series([], Name: count, dtype: int64)

In [20]:
df_variations["variation_inventory_id"].value_counts()

variation_inventory_id
KGZZ89270    1
CPRQ90324    1
NWDU90934    1
LROT89023    1
QHYZ00893    1
KHFD90751    1
UORD89925    1
XDIY00082    1
QNYE96569    1
SAGY91768    1
NYZA98888    1
TGWN96650    1
GGHX00903    1
MDDX95988    1
VRME00762    1
LDAC97015    1
IIKC90307    1
TZFM91331    1
RWRR30846    1
OQSJ90492    1
MMFT30494    1
DGMW97350    1
LLDE88401    1
VHRN90158    1
FMUV90638    1
NJXA90250    1
ACBX96049    1
AGYY89844    1
GNAY89939    1
WGDP88625    1
SSIH91035    1
WCTG88354    1
KKGW88137    1
VLWK91251    1
WYGR96303    1
Name: count, dtype: int64

In [21]:
id = "UYFJ61970"
xpto = df_variations[df_variations["variation_inventory_id"] == id]
xpto

,ml_code,inventory_id,variation_id,value_name,variation_inventory_id


In [22]:
id = "MLB2986538160"
xpto = df_variations[df_variations["ml_code"] == id]
xpto

,ml_code,inventory_id,variation_id,value_name,variation_inventory_id


In [23]:
df_variations["inventory_id"].value_counts()

Series([], Name: count, dtype: int64)

In [24]:
df_variations["value_name"].value_counts()

value_name
Preto                        6
Natural                      4
110/220                      3
Black night                  2
Destro                       2
Sunburst                     2
Vermelho-Verde               1
110V/220V                    1
Marrom                       1
Prateado                     1
PRATA                        1
Blue Reef Gradation          1
110V                         1
White                        1
BMC: Blue Metal Chameleon    1
Black flat                   1
Transparent red burst        1
C                            1
Preta                        1
Walnut flat                  1
Mahogany oil                 1
Drop sunburst satin          1
Name: count, dtype: int64

### Unindo as duas tabelas

In [25]:
# condf(df_sku_var, 'variation_inventory_id', '52181329446')

In [26]:
df_sku_var = pd.merge(
    df_sku,
    df_variations,
    left_on=["ml_code", "inventory_id"],
    right_on=["ml_code", "inventory_id"],
    how="left",
)
df_sku_var = df_sku_var.drop(["variations"], axis=1)
df_sku_var

,ml_code,inventory_id,status,catalog_listing,variation_id,value_name,variation_inventory_id
0,MLB2077624899,None,active,False,1.738839e+11,Vermelho-Verde,KGZZ89270
1,MLB2077566813,GOYP17092,active,False,NaN,NaN,NaN
2,MLB3642965726,GXOM30321,active,False,NaN,NaN,NaN
3,MLB3321195945,None,paused,False,1.777389e+11,Preto,RWRR30846
4,MLB3321186153,DGMW97350,active,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...
102,MLB4272236376,CMPE00886,active,True,NaN,NaN,NaN
103,MLB3532357445,BIXC96031,active,True,NaN,NaN,NaN
104,MLB3532268455,LKQN96991,active,True,NaN,NaN,NaN
105,MLB4275596684,IFOR00903,active,True,NaN,NaN,NaN


#### *if variation_inventory_id = None -> variation_inventory_id == inventory_id && remove inventory_id && variation_inventory_id rename to inventory_id*


In [27]:
df_sku_var["variation_inventory_id"].fillna(df_sku_var["inventory_id"], inplace=True)

# verificando se união funcionou
result = df_sku_var[df_sku_var["variation_inventory_id"] == df_sku_var["inventory_id"]]
result.sample(2)

,ml_code,inventory_id,status,catalog_listing,variation_id,value_name,variation_inventory_id
81,MLB3515842729,AEYS88396,active,True,NaN,NaN,AEYS88396
25,MLB3468571827,WGDP88625,active,True,NaN,NaN,WGDP88625


In [28]:
df_sku_var.columns

Index(['ml_code', 'inventory_id', 'status', 'catalog_listing', 'variation_id',
       'value_name', 'variation_inventory_id'],
      dtype='object')

In [29]:
# df_sku_var = df_sku_var.drop(['inventory_id'], axis=1)
cols = [
    "ml_code",
    "variation_inventory_id",
    "value_name",
    "variation_id",
    "status",
    "catalog_listing",
]
df_sku_var = df_sku_var[cols]
df_sku_var = df_sku_var.rename(columns={"variation_inventory_id": "inventory_id"})

print(f"Tamanho do dataframe final: {df_sku_var.shape}")
df_sku_var

Tamanho do dataframe final: (107, 6)


,ml_code,inventory_id,value_name,variation_id,status,catalog_listing
0,MLB2077624899,KGZZ89270,Vermelho-Verde,1.738839e+11,active,False
1,MLB2077566813,GOYP17092,NaN,NaN,active,False
2,MLB3642965726,GXOM30321,NaN,NaN,active,False
3,MLB3321195945,RWRR30846,Preto,1.777389e+11,paused,False
4,MLB3321186153,DGMW97350,NaN,NaN,active,True
...,...,...,...,...,...,...
102,MLB4272236376,CMPE00886,NaN,NaN,active,True
103,MLB3532357445,BIXC96031,NaN,NaN,active,True
104,MLB3532268455,LKQN96991,NaN,NaN,active,True
105,MLB4275596684,IFOR00903,NaN,NaN,active,True


In [30]:
# tratando variation_id
df_sku_var["variation_id"] = (
    df_sku_var["variation_id"].astype(str).apply(lambda x: x.rstrip(".0"))
)
df_sku_var["variation_id"] = df_sku_var["variation_id"].replace("nan", np.nan)
df_sku_var

,ml_code,inventory_id,value_name,variation_id,status,catalog_listing
0,MLB2077624899,KGZZ89270,Vermelho-Verde,173883881289,active,False
1,MLB2077566813,GOYP17092,NaN,NaN,active,False
2,MLB3642965726,GXOM30321,NaN,NaN,active,False
3,MLB3321195945,RWRR30846,Preto,177738888404,paused,False
4,MLB3321186153,DGMW97350,NaN,NaN,active,True
...,...,...,...,...,...,...
102,MLB4272236376,CMPE00886,NaN,NaN,active,True
103,MLB3532357445,BIXC96031,NaN,NaN,active,True
104,MLB3532268455,LKQN96991,NaN,NaN,active,True
105,MLB4275596684,IFOR00903,NaN,NaN,active,True


In [31]:
df_sku_var["variation_id"].value_counts()

variation_id
173883881289    1
178935162362    1
180171654729    1
17891952166     1
180194464367    1
180193075635    1
178933744228    1
180196117171    1
17894186426     1
178918886134    1
178941121678    1
178941126876    1
180955702667    1
180962069727    1
180962797063    1
180962355359    1
178918740762    1
178919001716    1
177738888404    1
178912001332    1
178271790957    1
178271891283    1
178909450832    1
180159251515    1
180158472193    1
180158464503    1
178912495204    1
17891182093     1
180159178631    1
180159176895    1
180158385571    1
17891191665     1
180158036941    1
180158032957    1
180963005163    1
Name: count, dtype: int64

In [32]:
df_sku_var.dtypes

ml_code            object
inventory_id       object
value_name         object
variation_id       object
status             object
catalog_listing      bool
dtype: object

In [33]:
condf(df_sku_var, "ml_code", "MLB1465231671")

,ml_code,inventory_id,value_name,variation_id,status,catalog_listing


#### verificações

In [34]:
count_nul = df_sku_var["inventory_id"].isna().sum()
rows_with_nulls = df_sku_var.loc[df_sku_var["inventory_id"].isna()]
print(df_sku_var.shape)
df_sku_var["inventory_id"].value_counts()
count_nul
rows_with_nulls  # fogão cocktop

(107, 6)


,ml_code,inventory_id,value_name,variation_id,status,catalog_listing


In [35]:
# # Mostrar e contar valores únicos em todas as colunas
for coluna in df_sku_var.columns:
    unique_values = df_sku_var[coluna].unique()
    count_unique = df_sku_var[coluna].nunique()
    print(f"Valores únicos na coluna '{coluna}': {unique_values}")
    print(f"Número de valores únicos na coluna '{coluna}': {count_unique}")
    print("\n-----------------------------\n")

Valores únicos na coluna 'ml_code': ['MLB2077624899' 'MLB2077566813' 'MLB3642965726' 'MLB3321195945'
 'MLB3321186153' 'MLB3321181307' 'MLB3321175509' 'MLB3321160629'
 'MLB3321122477' 'MLB3661055492' 'MLB3661047876' 'MLB4097062232'
 'MLB3467798335' 'MLB4099722814' 'MLB4099639536' 'MLB4099590706'
 'MLB4099564966' 'MLB4099421026' 'MLB4099400438' 'MLB4099382194'
 'MLB4099379150' 'MLB4099376828' 'MLB4099364154' 'MLB3468588713'
 'MLB3468588049' 'MLB3468571827' 'MLB3468508903' 'MLB3468507631'
 'MLB3468483295' 'MLB3468445013' 'MLB3468355335' 'MLB3468342661'
 'MLB3468337879' 'MLB3468334819' 'MLB3468308899' 'MLB3468277673'
 'MLB3468270317' 'MLB3468268905' 'MLB3468264183' 'MLB3468232887'
 'MLB4102601126' 'MLB4102433058' 'MLB4102407438' 'MLB4102391716'
 'MLB4102387026' 'MLB4102304362' 'MLB4102267624' 'MLB3469928691'
 'MLB3469801625' 'MLB3469766071' 'MLB3469737689' 'MLB3469722247'
 'MLB3469668577' 'MLB3469635843' 'MLB3469580967' 'MLB4107101184'
 'MLB4107062926' 'MLB4107047820' 'MLB4107046348' 'MLB4

### Populando banco de dados

In [36]:
df_sku_var.columns

Index(['ml_code', 'inventory_id', 'value_name', 'variation_id', 'status',
       'catalog_listing'],
      dtype='object')

In [37]:
df_sku_var.dtypes

ml_code            object
inventory_id       object
value_name         object
variation_id       object
status             object
catalog_listing      bool
dtype: object

In [38]:
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in df_sku_var.iterrows():
    # insert_query = sql.SQL(
    #     "INSERT INTO items_bueno (ml_code, inventory_id, value_name, variation_id, status, catalog_listing) VALUES (%s, %s, %s, %s, %s, %s)"
    # )
    insert_query = sql.SQL(
        "INSERT INTO items_cris (ml_code, inventory_id, value_name, variation_id, status, catalog_listing) VALUES (%s, %s, %s, %s, %s, %s)"
    )
    cursor.execute(
        insert_query,
        (
            row["ml_code"],
            row["inventory_id"],
            row["value_name"],
            row["variation_id"],
            row["status"],
            row["catalog_listing"],
        ),
    )

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print("Dados inseridos com sucesso!")

Dados inseridos com sucesso!


### Buscando itens no banco de dados

In [42]:
# ler tabela do db
try:
    conn = psycopg2.connect(**db_config)

    query = "SELECT * FROM items_bueno;"
    query = "SELECT * FROM items_cris;"
    # query = "SELECT * FROM items_mcenter;"
    df_items = pd.read_sql(query, conn)
except psycopg2.Error as e:
    print(f"Erro do psycopg2 em 'items': {e}")
    # TODO log
except Exception as e:
    print(f"Erro ao consultar 'items': {e}")
    # TODO log

print(df_items.shape)

(107, 8)


C:\Users\couti\AppData\Local\Temp\ipykernel_7032\1324128418.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_items = pd.read_sql(query, conn)


In [43]:
df_items

,ml_code,inventory_id,value_name,variation_id,status,catalog_listing,created_at,updated_at
0,MLB2077624899,KGZZ89270,Vermelho-Verde,173883881289,active,False,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
1,MLB2077566813,GOYP17092,NaN,NaN,active,False,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
2,MLB3642965726,GXOM30321,NaN,NaN,active,False,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
3,MLB3321195945,RWRR30846,Preto,177738888404,paused,False,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
4,MLB3321186153,DGMW97350,NaN,NaN,active,True,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
...,...,...,...,...,...,...,...,...
102,MLB4272236376,CMPE00886,NaN,NaN,active,True,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
103,MLB3532357445,BIXC96031,NaN,NaN,active,True,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
104,MLB3532268455,LKQN96991,NaN,NaN,active,True,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254
105,MLB4275596684,IFOR00903,NaN,NaN,active,True,2023-12-12 00:19:32.931254,2023-12-12 00:19:32.931254


In [ ]:
dx = df_items.copy()
dy = df_sku_var.copy()

In [ ]:
## remove linhas de data
dx = dx.drop(columns=["created_at", "updated_at"])

In [ ]:
dx.replace("NaN", np.nan, inplace=True)  # altera de strin para NaN
dx = dx.astype(str)
dy = dy.astype(str)

In [ ]:
dy.sample()

### Atualizar itens no banco de dados

In [ ]:
# Realize a junção (merge) com base nas colunas ml_code e inventory_id
merged_df = pd.merge(
    dy, dx, on=["ml_code", "inventory_id"], how="inner", suffixes=("_sku_var", "_items")
)

# Identifique as linhas com valores diferentes
different_rows = merged_df[
    (merged_df["value_name_sku_var"] != merged_df["value_name_items"])
    | (merged_df["status_sku_var"] != merged_df["status_items"])
    | (merged_df["catalog_listing_sku_var"] != merged_df["catalog_listing_items"])
]

# Exiba as linhas com valores diferentes
different_rows

In [ ]:
# Compare os DataFrames
identicos = dx.equals(dy)

# Exiba o resultado
print("Os DataFrames são idênticos:", identicos)

In [ ]:
# diferencas = df_db.compare(df_new)
diferencas = dx.compare(dy)

# Exiba as diferenças
print("Diferenças entre os DataFrames:")
diferencas

In [ ]:
# Encontrar diferenças usando merge
diferencas = (
    pd.merge(dx, dy, how="outer", indicator=True)
    .query('_merge == "left_only"')
    .drop("_merge", axis=1)
)

# Criar um novo DataFrame apenas com as colunas modificadas
df_atualizado = dx.copy()
df_atualizado[diferencas.columns] = diferencas

# Exibir DataFrame atualizado
print("Novo DataFrame Atualizado:")
df_atualizado

In [ ]:
# Remover linhas onde todos os valores em TODAS as colunas são NaN
df_atualizado_sem_nan = df_atualizado.dropna(how="all", subset=df_atualizado.columns)

# Exibir DataFrame atualizado
print("DataFrame Atualizado sem Linhas com Todos os Valores em Todas as Colunas NaN:")
df_atualizado_sem_nan

In [ ]:
try:
    # Conectar ao banco de dados
    conn = psycopg2.connect(**db_config)

    # Criar um cursor
    cursor = conn.cursor()

    # Iterar sobre as linhas do DataFrame e executar as atualizações no banco de dados
    for index, row in df_atualizado_sem_nan.iterrows():
        ml_code = row["ml_code"]
        inventory_id = row["inventory_id"]
        value_name = row["value_name"]
        variation_id = row["variation_id"]
        status = row["status"]
        catalog_listing = row["catalog_listing"]
        updated_at = datetime.now()  # Use a data/hora atual

        # Construir a instrução SQL de atualização
        update_query = sql.SQL(
            "UPDATE items_bueno SET value_name = %s, variation_id = %s, status = %s, catalog_listing = %s, updated_at = %s WHERE ml_code = %s AND inventory_id = %s"
        )

        # Executar a instrução SQL
        cursor.execute(
            update_query,
            (
                value_name,
                variation_id,
                status,
                catalog_listing,
                updated_at,
                ml_code,
                inventory_id,
            ),
        )

    # Commit para salvar as alterações
    conn.commit()

    print("Dados atualizados com sucesso!")

except psycopg2.Error as e:
    # Tratar erros específicos do PostgreSQL
    print(f"Erro ao executar operação no PostgreSQL: {e}")

except Exception as e:
    # Tratar outros erros
    print(f"Erro não esperado: {e}")

finally:
    # Fechar cursor e conexão mesmo se ocorrerem exceções
    if "cursor" in locals() and cursor is not None:
        cursor.close()

    if "conn" in locals() and conn is not None:
        conn.close()

### Popule com novos itens

In [ ]:
data = {
    "ml_code": ["MLBXXXXX"],
    "inventory_id": ["xxxxx"],
    "value_name": ["xxxx"],
    "variation_id": ["xxxx"],
    "status": ["xxxx"],
    "catalog_listing": [False],
}

df_ficticio = pd.DataFrame(data)

In [ ]:
# Encontrar linhas onde os pares ml_code e inventory_id em df_ficticio são diferentes de dx
diferenca = pd.merge(
    dx, df_ficticio, on=["ml_code", "inventory_id"], how="right", indicator=True
)

# Filtrar apenas as linhas em que df_ficticio tem valores diferentes de dx
diferenca = diferenca.query('_merge == "right_only"').drop(columns="_merge")
diferenca

In [ ]:
# Selecionar colunas específicas e renomear
diferenca = diferenca[
    [
        "ml_code",
        "inventory_id",
        "value_name_y",
        "variation_id_y",
        "status_y",
        "catalog_listing_y",
    ]
]
diferenca = diferenca.rename(
    columns={
        "value_name_y": "value_name",
        "status_y": "status",
        "catalog_listing_y": "catalog_listing",
    }
)
diferenca

In [ ]:
# # Encontrar linhas onde os pares ml_code e inventory_id em dy são diferentes de dx
# linhas_diferentes = pd.merge(dx, df_ficticio, on=['ml_code', 'inventory_id'], how='right', indicator=True).query('_merge == "right_only"').drop('_merge', axis=1)

# # Exibir as linhas diferentes
# print("Linhas onde ml_code e inventory_id em dy são diferentes de dx:")
# linhas_diferentes

In [ ]:
# # Encontrar linhas que existem em dy, mas não em dx
# linhas_novas = pd.merge(dx, df_ficticio,on=['ml_code', 'inventory_id'], how='right', indicator=True).query('_merge == "right_only"').drop('_merge', axis=1)

# # Exibir as linhas novas
# print("Linhas que existem em dy, mas não em dx:")
# linhas_novas

In [ ]:
# Inserir novos dados no banco de dados
conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in diferenca.iterrows():
    insert_query = sql.SQL(
        "INSERT INTO items (ml_code, inventory_id, value_name, status, catalog_listing) VALUES (%s, %s, %s, %s, %s)"
    )
    cursor.execute(
        insert_query,
        (
            row["ml_code"],
            row["inventory_id"],
            row["value_name"],
            row["status"],
            row["catalog_listing"],
        ),
    )

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print("Dados inseridos com sucesso!")